In [2]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.regularizers import l2,l1_l2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import keras.preprocessing.image as image
from tensorflow.keras.applications import VGG16

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2, horizontal_flip=True,vertical_flip=True,
                                   fill_mode="nearest", rotation_range=10,brightness_range=[0.8,1.2],width_shift_range=0.1, height_shift_range=0.1)
validation_datagen = ImageDataGenerator(rescale=1./255)
direc='C:\\Users\\HP\\.vscode\\Python\\AI\\ML Projects\\Datasets\\Apple\\'
Train=train_datagen.flow_from_directory(direc+'train',target_size=(64,64),batch_size=32,class_mode='binary')
Validate=validation_datagen.flow_from_directory(direc+'valid',target_size=(64,64),batch_size=32,class_mode='binary')

Found 4024 images belonging to 2 classes.
Found 1006 images belonging to 2 classes.


In [6]:

# Load the pre-trained VGG16 model
vgg_model = VGG16(input_shape=(64, 64, 3), include_top=False, weights='imagenet')

# Freezing some layers so that there weights won't be updated
for layer in vgg_model.layers:
    layer.trainable = False
for layer in vgg_model.layers[-4:]:
    layer.trainable = True

# Check which layers are trainable
for layer in vgg_model.layers:
    print(layer, layer.trainable)

# Create a custom classification head
model = Sequential([
    vgg_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

<keras.src.engine.input_layer.InputLayer object at 0x000001A22BA886D0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x000001A22BFF14F0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x000001A22BFF1CD0> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x000001A22C047D00> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x000001A22C047CD0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x000001A22C088730> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x000001A22C0A3850> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x000001A22C088C40> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x000001A22C0AC2E0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x000001A22C0B8460> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x000001A22C0BC7F0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x000001A22C0B8520> False


In [7]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001, verbose=1)
checkpoint=ModelCheckpoint('best_model.h5',monitor='val_loss',mode='min',save_best_only=True,verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

In [9]:
history1=model.fit(x=Train,validation_data=Validate,epochs=10,callbacks=[early_stopping,checkpoint,reduce_lr],verbose=1)

Epoch 1/10
126/126 [==============================] - ETA: 0s - loss: 0.1162 - accuracy: 0.9573
Epoch 1: val_loss did not improve from 0.06102
126/126 [==============================] - 111s 884ms/step - loss: 0.1162 - accuracy: 0.9573 - val_loss: 0.0719 - val_accuracy: 0.9682 - lr: 1.0000e-04
Epoch 2/10
126/126 [==============================] - ETA: 0s - loss: 0.0929 - accuracy: 0.9652
Epoch 2: val_loss improved from 0.06102 to 0.04785, saving model to best_model.h5
126/126 [==============================] - 113s 897ms/step - loss: 0.0929 - accuracy: 0.9652 - val_loss: 0.0479 - val_accuracy: 0.9831 - lr: 1.0000e-04
Epoch 3/10
126/126 [==============================] - ETA: 0s - loss: 0.0862 - accuracy: 0.9665
Epoch 3: val_loss did not improve from 0.04785
126/126 [==============================] - 105s 832ms/step - loss: 0.0862 - accuracy: 0.9665 - val_loss: 0.0547 - val_accuracy: 0.9742 - lr: 1.0000e-04
Epoch 4/10
126/126 [==============================] - ETA: 0s - loss: 0.0749 - a